# 使用 LangChain 实现本地 Agent

## 简介

ReAct（Reasoning and Acting）是一种将推理与行动相结合的框架，用于增强智能体在复杂任务中的表现。该框架通过将逻辑推理与实际行动紧密结合，使智能体能够在动态环境中更有效地完成任务。


![](https://datawhalechina.github.io/handy-ollama/images/C7-5-1.png)


本文档介绍了如何使用 ReAct 框架在 Ollama 中实现本地代理（Agent）。通过结合 Ollama 的功能与 ReAct 的灵活性，用户能够在本地环境中创建一个高效的交互式代理。此实现能够处理复杂任务，支持多种交互模式，并且优化了任务自动化和用户体验，适合需要高实时性的本地应用场景。




## 导入依赖





In [23]:
from langchain_core.tools import tool  
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import render_text_description
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain_community.chat_models import ChatOllama

## 初始化 Agent 工具

定义 SearchInput 类继承 BaseModel，用于定义输入数据的模式。

@tool(args_schema=SearchInput) 使用工具装饰器装饰 weather_forecast 函数，并指定其输入模式为 SearchInput。

In [70]:
# ============================================================
# 自定义工具
# ============================================================
import datetime


class SearchInput(BaseModel):
    location: str = Field(description="Weather for location")  # 定义一个 Pydantic 模型，用于描述输入模式，并提供描述信息

@tool(args_schema=SearchInput)
def weather_forecast(location: str):
    """Weather for location to search for"""
    print(f"Weather for {location}")  # 打印要预报天气的位置
    # return f"A dummy forecast for {location}."  # 返回给定位置的虚拟天气预报
    return f" {location} 下大雨."  # 返回给定位置的虚拟天气预报


@tool
def current_datetime(input: str) -> str:  # Note: A tool always needs an input and returns an output
    """Get the current date and time

    Returns:
        str: The current date and time
    """
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%A %d %B %Y, %I:%M%p')
    return formatted_time

print("weather_forecast", weather_forecast(''))
print("current_datetime", current_datetime(''))

Weather for 
weather_forecast   下大雨.
current_datetime Saturday 22 March 2025, 08:51PM


## 本地运行

在本例中使用 gemma:2b 模型，对于不同类型的模型，输出结果可能会很不一样（随机性比较大）。

In [ ]:
# 测试不加工具
llm = ChatOllama(model="gemma2:2b")  # 初始化 ChatOllama 模型，使用 "gemma2:2b"
llm.invoke("What is the weather in Paris?").content 

'I do not have access to real-time information, including weather updates. \n\nTo get the current weather in Paris, I recommend checking a reliable weather website or app such as:\n\n* **Google Weather:** Just search "weather in Paris" on Google.\n* **AccuWeather:** https://www.accuweather.com/\n* **The Weather Channel:** https://weather.com/ \n\n\nHave a wonderful time in Paris!  😊\n'

## hwchase17/react-json




In [55]:
hwchase17_react_json_prompt ='''
Answer the following questions as best you can. You have access to the following tools:

{tools}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: {tool_names}

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding.

Question: {input}
Thought:{agent_scratchpad}
'''

template = """ 
You are a great AI-Assistant that has access to additional tools in order to answer the following questions as best you can. Always answer in the same language as the user question. You have access to the following tools:

{tools}

To use a tool, please use the following format:

'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 3 times)
'''

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
'''
Thought: Do I need to use a tool? No
Final Answer: [your response here]
'''


Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

In [71]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# 测试使用工具
# llm = ChatOllama(model="gemma2:2b")  # 初始化 ChatOllama 模型，使用 "gemma2:2b"
llm = ChatOllama(model="llama3.2")  # 初始化 ChatOllama 模型，使用 "llama3.2"
tools = [weather_forecast, current_datetime]  # 使用 weather_forecast 工具
# prompt = hub.pull("hwchase17/react-json")  # 从 hub 拉取特定提示
# prompt = PromptTemplate.from_template(hwchase17_react_json_prompt)
prompt = PromptTemplate.from_template(template)
# prompt = prompt.partial(
#     tools=render_text_description(tools),  # 为提示呈现工具的文本描述
#     tool_names=", ".join([t.name for t in tools]),  # 将工具名称连接成一个以逗号分隔的字符串
# )

# print("tools", render_text_description(tools))
# print("tool_names", ", ".join([t.name for t in tools]))

agent = create_react_agent(llm, tools, prompt)  # 使用 llm、工具和自定义提示创建代理
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)  # 使用指定参数初始化 AgentExecutor






In [72]:

user_prompt = "How are you?"
response = agent_executor.invoke(
    {"input": user_prompt}
)
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: current_datetime
Action Input: Saturday 22 March 2025, 08:51PMThought: Do I need to use a tool? No
Final Answer: I am functioning within normal parameters and ready to assist you.

> Finished chain.
I am functioning within normal parameters and ready to assist you.


In [73]:
user_prompt="Get the current date and time?"
response = agent_executor.invoke(
    {"input": user_prompt}
)
print(response["output"])




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: current_datetime
Action Input: Saturday 22 March 2025, 08:51PMThought: Do I need to use a tool? No
Final Answer: Saturday 22 March 2025, 08:51PM

> Finished chain.
Saturday 22 March 2025, 08:51PM


In [74]:
user_prompt="What is the weather in New York?"
response = agent_executor.invoke(
    {"input": user_prompt}
)
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: weather_forecast
Action Input: New YorkWeather for New York
 New York 下大雨.Thought: Do I need to use a tool? No
Final Answer: The current weather in New York is experiencing heavy rain.

> Finished chain.
The current weather in New York is experiencing heavy rain.


## 使用对话历史

在使用对话历史时，需要使用 react-chat Prompt 模板。在 invoke 时，加入 chat_history。

In [19]:
# 拉去特定提示，注意此处使用的是 react-chat
prompt = hub.pull("hwchase17/react-chat")

# 构建 ReAct agent
agent_history = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent_history, tools=tools, verbose=False)

agent_executor.invoke(
    {
        "input": "what's my name? Only use a tool if needed, otherwise respond with Final Answer",
        "chat_history": "Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you",
    }
)


Weather for Your Location


{'input': "what's my name? Only use a tool if needed, otherwise respond with Final Answer",
 'chat_history': 'Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you',
 'output': 'Nice to meet you too, Bob! How can I help you today? \n```'}